In [ ]:
grm_inventory = dict([x.strip().split() for x in open(grm_list).readlines() if x.strip() and not x.strip().startswith('#')])

def get_grm_file(chrom, grm_list, grm_inventory):
    chrom = f'{chrom}'
    if chrom.startswith('chr'):
        chrom = chrom[3:]
    if chrom not in grm_inventory:
        grm_file = f'{chrom}'
    else:
        grm_file = grm_inventory[chrom]
    if not os.path.isfile(grm_file):
        # relative path
        if not os.path.isfile(f'{grm_list:ad}/' + grm_file):
            raise ValueError(f"Cannot find grm file {grm_file}")
        else:
            grm_file = f'{grm_list:ad}/' + grm_file
    return path(grm_file)



In [5]:
a = read_delim("~/GIT/GEUVADIS.445_samples.covariates.txt",delim = "\t")

Warning message:
“Missing column names filled in: 'X1' [1]”

── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  X1 = col_character()
)
ℹ Use `spec()` for the full column specifications.




In [1]:
library("dplyr")
library("tibble")
library("readr")
library("purrr")
library("modelr")
library("tidyr")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [6]:
a

X1,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00105,HG00106,⋯,NA20809,NA20810,NA20811,NA20812,NA20813,NA20814,NA20815,NA20819,NA20826,NA20828
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
PC1,-0.024000000,-0.0243000000,-0.024900000,-0.022100000,-0.024000000,-2.490000e-02,-0.023700000,-0.0236000000,-0.0247000000,⋯,-0.019700000,-0.020800000,-0.020100000,-0.0222000000,-0.021100000,-2.100000e-02,-0.020600000,-0.0197000000,-0.0212000000,-0.020600000
PC2,-0.014500000,-0.0212000000,-0.016200000,-0.030300000,-0.014800000,-1.160000e-02,-0.013700000,-0.0154000000,-0.0242000000,⋯,-0.068700000,-0.066000000,-0.061400000,-0.0597000000,-0.060900000,-5.850000e-02,-0.064900000,-0.0585000000,-0.0659000000,-0.069900000
PC3,0.004700000,0.0296000000,0.032900000,-0.040800000,0.014900000,1.980000e-02,0.008400000,0.0291000000,0.0316000000,⋯,0.031500000,0.030500000,0.029800000,0.0367000000,0.036300000,3.390000e-02,0.032000000,0.0285000000,0.0336000000,0.036400000
PC4,0.072400000,0.0861000000,0.101800000,0.029600000,0.098900000,8.380000e-02,0.070900000,0.0944000000,0.0620000000,⋯,-0.056700000,-0.064000000,-0.053200000,-0.0504000000,-0.038800000,-5.350000e-02,-0.045900000,-0.0558000000,-0.0573000000,-0.047800000
PC5,0.027300000,-0.0020000000,-0.006200000,0.011700000,0.030300000,2.290000e-02,0.005400000,-0.0013000000,0.0051000000,⋯,0.008200000,-0.003800000,0.005900000,-0.0205000000,-0.013000000,-5.500000e-03,-0.003800000,0.0061000000,0.0024000000,-0.001600000
InferredCov1,0.012724601,0.0221059844,0.015363757,0.013403445,0.025456475,6.688620e-02,0.005413158,-0.0308675822,-0.0017858674,⋯,-0.050657291,-0.054405134,0.010820721,-0.0142917642,-0.010465370,9.587055e-03,-0.027488519,0.0173024647,-0.0286082961,0.019722985
InferredCov2,-0.030224379,-0.0208609626,0.031485394,0.008697151,-0.008585763,6.080864e-02,0.002658807,0.0495678261,0.0133153889,⋯,-0.024989497,0.023815652,-0.020491857,0.0113718528,0.019766923,5.179050e-02,0.056529116,-0.0320266187,0.0070663705,0.024832608
InferredCov3,0.008843166,-0.0001722985,-0.034724459,0.008554962,-0.013174459,2.406089e-02,0.076760732,-0.0001566004,0.0378046334,⋯,-0.003838480,0.009051641,-0.011320127,-0.0311438069,0.006699343,-1.397638e-02,-0.052956346,0.0305750147,-0.0078386236,0.056545477
InferredCov4,0.026157659,0.0171643179,-0.024749078,-0.008636067,0.125545964,2.905327e-02,0.014539688,0.0057089948,-0.0004756877,⋯,0.053535659,0.036316916,0.074651964,-0.0051851398,-0.013089463,1.980858e-02,-0.037104394,-0.0002824374,0.0576519221,-0.085695699


In [8]:
w  = read_delim("~/python.txt",delim = "\t")


── Column specification ────────────────────────────────────────────────────────
cols(
  phenotype_id = col_character(),
  num_var = col_double(),
  beta_shape1 = col_double(),
  beta_shape2 = col_double(),
  true_df = col_double(),
  pval_true_df = col_double(),
  variant_id = col_character(),
  tss_distance = col_double(),
  ma_samples = col_double(),
  ma_count = col_double(),
  af = col_double(),
  pval_nominal = col_double(),
  slope = col_double(),
  slope_se = col_double(),
  pval_perm = col_double(),
  pval_beta = col_double()
)




In [3]:
2

[1] 2

In [9]:
w

phenotype_id,num_var,beta_shape1,beta_shape2,true_df,pval_true_df,variant_id,tss_distance,ma_samples,ma_count,af,pval_nominal,slope,slope_se,pval_perm,pval_beta
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000142149,6824,1.002532,576.0943,555.1693,2.971194e-07,21:30876478_A_G,-996543,15,15,0.011295181,2.462257e-08,1.027954e-01,1.821086e-02,0.00049995,0.0001672575
ENSG00000141956,8752,1.031421,1112.9819,575.7334,3.298047e-04,21:42717962_A_G,919736,15,15,0.011295181,1.250792e-04,5.640861e-05,1.461737e-05,0.29107089,0.2928872158
ENSG00000141959,8862,1.007035,1085.4427,571.8396,2.464477e-04,21:44935449_C_T,635397,12,12,0.009036144,8.507073e-05,-6.852548e-02,1.732905e-02,0.23167683,0.2317672491


In [10]:
w

phenotype_id,num_var,beta_shape1,beta_shape2,true_df,pval_true_df,variant_id,tss_distance,ma_samples,ma_count,af,pval_nominal,slope,slope_se,pval_perm,pval_beta
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000142149,6824,1.002532,576.0943,555.1693,2.971194e-07,21:30876478_A_G,-996543,15,15,0.011295181,2.462257e-08,1.027954e-01,1.821086e-02,0.00049995,0.0001672575
ENSG00000141956,8752,1.031421,1112.9819,575.7334,3.298047e-04,21:42717962_A_G,919736,15,15,0.011295181,1.250792e-04,5.640861e-05,1.461737e-05,0.29107089,0.2928872158
ENSG00000141959,8862,1.007035,1085.4427,571.8396,2.464477e-04,21:44935449_C_T,635397,12,12,0.009036144,8.507073e-05,-6.852548e-02,1.732905e-02,0.23167683,0.2317672491


In [12]:
colnames(w)[2:3] = c("a","b")

In [13]:
w

phenotype_id,a,b,beta_shape2,true_df,pval_true_df,variant_id,tss_distance,ma_samples,ma_count,af,pval_nominal,slope,slope_se,pval_perm,pval_beta
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000142149,6824,1.002532,576.0943,555.1693,2.971194e-07,21:30876478_A_G,-996543,15,15,0.011295181,2.462257e-08,1.027954e-01,1.821086e-02,0.00049995,0.0001672575
ENSG00000141956,8752,1.031421,1112.9819,575.7334,3.298047e-04,21:42717962_A_G,919736,15,15,0.011295181,1.250792e-04,5.640861e-05,1.461737e-05,0.29107089,0.2928872158
ENSG00000141959,8862,1.007035,1085.4427,571.8396,2.464477e-04,21:44935449_C_T,635397,12,12,0.009036144,8.507073e-05,-6.852548e-02,1.732905e-02,0.23167683,0.2317672491
